## Kaggle Settings

In [ ]:
!pip install tokenizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/cgjeong23/Viral-genomic-classification.git virus

%cd virus

## Run

In [ ]:
use_kaggle = True

train_data_path = '../../input/pacific-sra/trainingdata' if use_kaggle else 'trainingdata'
save_path = '/kaggle/working' if use_kaggle else 'assets'

In [ ]:
import torch
import numpy as np
import pickle

from ML.inference import infer, load_for_inference
from ML.dataloader import load_sequences

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequence, label = load_sequences(train_data_path)

label_index = {k: [] for k in set(label)}

for i, l in enumerate(label):
    label_index[l].append(i)

label_index = {k: np.random.choice(v, 100000) for k, v in label_index.items()}

sampled_index = np.concatenate(list(label_index.values()))

sequence = [sequence[i] for i in sampled_index]
label = [label[i] for i in sampled_index]

model, tokenizer, label_dict = load_for_inference('ML/assets/model.pth',
                                                  'ML/assets/gene_tokenizer.json',
                                                  'ML/assets/label_dict.json')

model = model.to(device)

batch_size = 1000
num_batches = len(sequence) // batch_size
if len(sequence) % batch_size != 0:
    num_batches += 1

all_embeddings = []
for i in range(num_batches):
    emb = infer(sequence[i * batch_size: (i+1) * batch_size], 
                tokenizer, 
                model.embedding, 
                device=device)
    all_embeddings.append(emb)

all_embeddings = torch.cat(all_embeddings, 0).cpu().numpy()


np.save(f'{save_path}/virus_embeddings.npy', all_embeddings)
with open(f'{save_path}/label.pkl','wb') as f:
    pickle.dump(label, f)

### Do PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3) # reduce to 3d
pca.fit(all_embeddings)
embeddings_3d = pca.transform(all_embeddings)

np.save(f'{save_path}/virus_embeddings_3d.npy',embeddings_3d)

with open(f'{save_path}/virus_pca.pkl','wb') as f:
    pickle.dump(pca, f)